<h1>Gradient Boosting</h1>
<p>method composed of naive model and cycle</p>
<ol>
    <h2>naive model</h2>
    <li> initializing the ensemble with a single model.<br>it is ok that predictions of this sigle model are inaccuratee.</li>
    <h2> cycle </h2>
    <li> generate predictions for each observation with the current ensemble<br>predictions are used to calculate a loss function</li>
    <li>fit a new model that will be added to the ensemble by using loss function<br>examine and determine the parameters in order for new model added to ensemble to reduce the loss<br>we use <b>gradient descent</b> on this loss function to determine the parameters</li>
    <li>we add the new model to the ensemble</li>
    <li>repeat  2~4</li>
    </ol>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("melb_data.csv")
data.dropna(subset = ["Price"], axis = 0, inplace = True)
y_full = data.Price
X_full = data.drop("Price",axis = 1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full,y_full, train_size = 0.8, random_state = 0)
cols = ["Rooms",'Distance','Landsize','BuildingArea','YearBuilt']

X = data[cols]
y = data.Price

X_train, X_valid, y_train,y_valid = train_test_split(X,y,train_size =0.8, random_state=0)

<h1>XGBoost</h1>
<p><b>extreme gradient boosting</b></p>

In [3]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [4]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print( mean_absolute_error(predictions,y_valid))

236021.00835615335


<p>as avobe, XGBRegressor has a lot of paramter<br></p>

<h1>Parameter Tuning</h1>
<p>some example of paramters</p>
<ul>
    <li>n_estimators
        <ol>
            <li>haw many times to go through the modeling cycle / how many models added to ensemble</li>
            <li>if too low, underfitting</li>
            <li>if too high, overfitting</li> 
   </li>
    <li>learning_rate
        <ol>
            <li>before adding each predictions, we can multiply the predictions from each model by a small number(this number is learning_rate)</li>
    </li>
    <li>early_stopping_rounds
        <ol>
            <li>automatically find the ideal value for n_estimators.</li>
            <li> thanks to early stopping, the model stops iterating when the validation score stops improving </li>
            <li> usually it is good way to set high n_estimators and use early_stopping_rounds</li>
            <li> should use eval_set in useing early_stopping_rounds</li>
   </li>
    <li>n_jobs
        <ol>
            <li>on larger datasets, setting n_jobs is sutable</li>
            <li>the number of cores of CPU on the machine</li>
    
   

In [5]:
model1 = XGBRegressor(n_estimators = 500)
model1.fit(X_train, y_train)
preds1 = model1.predict(X_valid)
print(mean_absolute_error(preds1,y_valid))

248417.43546345728


In [6]:
model1 = XGBRegressor(n_estimators = 500)
model1.fit(X_train, y_train,early_stopping_rounds=5, eval_set = [(X_valid, y_valid)],verbose = False)
preds1 = model1.predict(X_valid)
print(mean_absolute_error(preds1,y_valid))

238594.2109892765


<p>verbose --- </p>

In [7]:
model1 = XGBRegressor(n_estimators = 500, learning_rate = 0.05)
model1.fit(X_train, y_train,early_stopping_rounds=5, eval_set = [(X_valid, y_valid)],verbose = False)
preds1 = model1.predict(X_valid)
print(mean_absolute_error(preds1,y_valid))

242422.7209131075


In [8]:
model1 = XGBRegressor(n_estimators = 10000, learning_rate = 0.05, n_jobs = 4)
model1.fit(X_train, y_train,early_stopping_rounds=5, eval_set = [(X_valid, y_valid)],verbose = False)
preds1 = model1.predict(X_valid)
print(mean_absolute_error(preds1,y_valid))

242422.7209131075
